# Tracing 101

Step through this notebook to understand how tracing works in Generative AI Toolkit.

The Generative AI Toolkit comes with these tracers out-of-the-box:


In [ ]:
from generative_ai_toolkit.tracer import (
    HumanReadableTracer,
    InMemoryTracer,
    IterableTracer,
    NoopTracer,
    SqliteTracer,
    StructuredLogsTracer,
    TeeTracer,
    Tracer,
)
from generative_ai_toolkit.tracer.dynamodb import DynamoDbTracer
from generative_ai_toolkit.tracer.otlp import OtlpTracer

In [3]:
# Other imports
import time

from generative_ai_toolkit.utils.ulid import Ulid

### `InMemoryTracer`

Use the in-memory tracer for testing and development:


In [4]:
in_memory_tracer = InMemoryTracer(
    memory_size=1000  # Store max 1000 traces, before discarding older ones
)

# Context, added to all traces:
in_memory_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with in_memory_tracer.trace("parent") as parent_span:
    parent_span.add_attribute("foo", "bar")
    parent_span.add_attribute(
        "inherited.foo",
        "bar",
        inheritable=True,  # Inheritable attributes propagate to child spans
    )
    time.sleep(0.1)

    # Nested spans become child spans, that point to the parent (parent_span_id):
    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("bar", "foo")
        time.sleep(0.1)

for trace in in_memory_tracer.get_traces():
    print(trace)
    print()

Trace(span_name='parent', span_kind='INTERNAL', trace_id='adcd3c555260da3929033531e821e89c', span_id='89c916d2d63aa46a', parent_span_id=None, started_at=datetime.datetime(2025, 7, 4, 11, 12, 48, 665922, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 48, 873655, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)

Trace(span_name='child', span_kind='INTERNAL', trace_id='adcd3c555260da3929033531e821e89c', span_id='608c44b632ecab87', parent_span_id='89c916d2d63aa46a', started_at=datetime.datetime(2025, 7, 4, 11, 12, 48, 768497, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 48, 873590, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'inherited.foo': 'bar'}, span_status='UNSET', resource_attributes={'service.name': 'MyAgent'}, scope=generative-ai-toolkit@current)



### Printing a human-readable version of traces during development

In the following example we add attributes that Generative AI Toolkit understands. It will use these to present traces in a way that is nicer to the human eye:


In [5]:
conversation_id = Ulid().ulid

with in_memory_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute(
        "ai.auth.context", {"principal_id": "user123"}, inheritable=True
    )
    time.sleep(0.1)

    with in_memory_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in in_memory_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,  # filter traces by conversation id
        "ai.subcontext.id": None
    }
):
    print(trace.as_human_readable())
    print()

[94e99f04b60cea27d7b0f744b4a17d3e/root/6c894f8009912f45] MyAgent SERVER 2025-07-04T11:12:48.882Z - parent
  ai.conversation.id=01JZAJ75QJP9K1BFZM8SJRBB1B ai.auth.context=user123


[94e99f04b60cea27d7b0f744b4a17d3e/6c894f8009912f45/681c5348bf73ae80] MyAgent INTERNAL 2025-07-04T11:12:48.987Z - child
  ai.trace.type=tool-invocation ai.conversation.id=01JZAJ75QJP9K1BFZM8SJRBB1B ai.auth.context=user123
         Input: Hello, world!
        Output: World, hello!




### `HumanReadableTracer`

You can also use the `HumanReadableTracer` that will log traces in human readable form to stdout, which is useful during development.

Note that traces are logged when the span ends, so parent spans are logged after child spans (this is true for all tracers):


In [6]:
import sys

human_readable_tracer = HumanReadableTracer(stream=sys.stdout)

human_readable_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with human_readable_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute(
        "ai.auth.context", {"principal_id": "user123"}, inheritable=True
    )
    time.sleep(0.1)

    with human_readable_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

[3b5a67be32c72869ce695281bdad26dd/fad7b98c935591ee/ba0711bbf950ec36] MyAgent INTERNAL 2025-07-04T11:12:49.212Z - child
  ai.trace.type=tool-invocation ai.conversation.id=01JZAJ75QJP9K1BFZM8SJRBB1B ai.auth.context=user123
         Input: Hello, world!
        Output: World, hello!

[3b5a67be32c72869ce695281bdad26dd/root/fad7b98c935591ee] MyAgent SERVER 2025-07-04T11:12:49.108Z - parent
  ai.conversation.id=01JZAJ75QJP9K1BFZM8SJRBB1B ai.auth.context=user123



### `StructuredLogsTracer`

Use the `StructuredLogsTracer` to log traces to stdout as JSON:


In [7]:
structured_logs_tracer = StructuredLogsTracer(stream=sys.stdout)

structured_logs_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with structured_logs_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute(
        "ai.auth.context", {"principal_id": "user123"}, inheritable=True
    )
    time.sleep(0.1)

    with structured_logs_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"child","span_kind":"INTERNAL","trace_id":"f2bb49eae88af55475fd98b3b7862ab3","span_id":"fef4a910c00969f3","parent_span_id":"ee2bf749c66fd317","started_at":"2025-07-04 11:12:49.431790+00:00","ended_at":"2025-07-04 11:12:49.535886+00:00","duration_ms":104,"attributes":{"ai.trace.type":"tool-invocation","ai.tool.input":"Hello, world!","ai.tool.output":"World, hello!","ai.conversation.id":"01JZAJ75QJP9K1BFZM8SJRBB1B","ai.auth.context":{"principal_id":"user123"}},"span_status":"UNSET","resource_attributes":{"service.name":"MyAgent"},"scope":{"name":"generative-ai-toolkit","version":"current"}}}
{"logger":"TraceLogger","level":"INFO","message":"Trace","trace":{"span_name":"parent","span_kind":"SERVER","trace_id":"f2bb49eae88af55475fd98b3b7862ab3","span_id":"ee2bf749c66fd317","parent_span_id":null,"started_at":"2025-07-04 11:12:49.330367+00:00","ended_at":"2025-07-04 11:12:49.536632+00:00","duration_ms":206,"attribu

### `DynamoDbTracer`

Use the `DynamoDbTracer` to store traces to DynamoDB.

To use this tracer, you should have created a table with partition key `pk` (string) and sort key `sk` (string).

If you want to support getting traces by conversation ID, the table must have a GSI with partition key `conversation_id` (string) and sort key `sk` (string).

For example, here's how to create such a table:


In [8]:
!aws dynamodb create-table \
  --table-name MyTracesTable \
  --attribute-definitions \
    AttributeName=pk,AttributeType=S \
    AttributeName=sk,AttributeType=S \
    AttributeName=conversation_id,AttributeType=S \
  --key-schema \
    AttributeName=pk,KeyType=HASH \
    AttributeName=sk,KeyType=RANGE \
  --billing-mode PAY_PER_REQUEST \
  --global-secondary-indexes '[{"IndexName":"conversation_index","KeySchema":[{"AttributeName":"conversation_id","KeyType":"HASH"},{"AttributeName":"sk","KeyType":"RANGE"}],"Projection":{"ProjectionType":"ALL"}}]'


An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: MyTracesTable


Then, use that table in the `DynamoDbTracer`:


In [9]:
conversation_id = Ulid().ulid
auth_context = {"principal_id": "user123"}

ddb_tracer = DynamoDbTracer(
    table_name="MyTracesTable",
    identifier="MyAgent",
    conversation_id_gsi_name="conversation_index",
)

ddb_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with ddb_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with ddb_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


for trace in ddb_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
        "ai.subcontext.id": None
    }
):
    print(trace.as_human_readable())
    print()

[146d4a9c8cdd77aee8dcfe4d3db41513/root/19871a56135db8ac] MyAgent SERVER 2025-07-04T11:12:54.586Z - parent
  ai.conversation.id=01JZAJ79NR857YAE89GF02WE2S ai.auth.context=user123


[146d4a9c8cdd77aee8dcfe4d3db41513/19871a56135db8ac/dc378eb1b7c257b8] MyAgent INTERNAL 2025-07-04T11:12:54.689Z - child
  ai.trace.type=tool-invocation ai.conversation.id=01JZAJ79NR857YAE89GF02WE2S ai.auth.context=user123
         Input: Hello, world!
        Output: World, hello!




### `OtlpTracer`

The `OtlpTracer` logs traces in Open Telemetry protobuf format. It expects you to run an Open Telemetry collector, that it can send the traces to. By default, it expects the collector to be run on localhost port 4318.

You can use the `OtlpTracer` to send traces to AWS X-Ray. To make that work, you can run the [ADOT collector](https://github.com/aws-observability/aws-otel-collector) locally:


In [10]:
# Create the ADOT config file:

yaml_content = """\
receivers:
  otlp:
    protocols:
      http:
        endpoint: 0.0.0.0:4318

processors:
  batch/traces:
    timeout: 10s
    send_batch_size: 50

exporters:
  awsxray:
    region: eu-central-1
    indexed_attributes:
      - ai.conversation.id

service:
  pipelines:
    traces:
      receivers: [otlp]
      processors: [batch/traces]
      exporters: [awsxray]
"""

with open("adot-config.yaml", "w") as f:
    f.write(yaml_content)

Run the ADOT collector in the background. Note that the following example assumes `AWS_REGION`, `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_SESSION_TOKEN` are available as environment variables:


In [11]:
!docker run --rm -d --name adot-collector \
  -p 4318:4318 \
  -e AWS_REGION \
  -e AWS_ACCESS_KEY_ID \
  -e AWS_SECRET_ACCESS_KEY \
  -e AWS_SESSION_TOKEN \
  -v $(pwd)/adot-config.yaml:/etc/collector-config.yaml \
  public.ecr.aws/aws-observability/aws-otel-collector:latest \
  --config=/etc/collector-config.yaml
!sleep 2 # wait for the collector to start
!docker logs adot-collector

2ebcfa9cba8fc84de6e28e07faa92232bd82f667277eb3b2f907554902d7b248
2025/07/04 11:12:55 ADOT Collector version: v0.43.2
2025/07/04 11:12:56 found no extra config, skip it, err: open /opt/aws/aws-otel-collector/etc/extracfg.txt: no such file or directory
2025/07/04 11:12:56 attn: users of the `datadog`, `logzio`, `sapm`, `signalfx` exporter components. please refer to https://github.com/aws-observability/aws-otel-collector/issues/2734 in regards to an upcoming ADOT Collector breaking change
2025-07-04T11:12:56.055Z	info	service@v0.117.0/service.go:164	Setting up own telemetry...
2025-07-04T11:12:56.058Z	info	telemetry/metrics.go:70	Serving metrics	{"address": "localhost:8888", "metrics level": "Normal"}
2025-07-04T11:12:56.092Z	info	service@v0.117.0/service.go:230	Starting aws-otel-collector...	{"Version": "v0.43.2", "NumCPU": 2}
2025-07-04T11:12:56.092Z	info	extensions/extensions.go:39	Starting extensions...
2025-07-04T11:12:56.094Z	info	otlpreceiver@v0.117.0/otlp.go:169	Starting HTTP ser

Then, send traces to AWS X-Ray by using the `OtlpTracer`:


In [12]:
otlp_tracer = OtlpTracer()

otlp_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with otlp_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", "123456", inheritable=True)
    parent_span.add_attribute(
        "ai.auth.context", {"principal_id": "user123"}, inheritable=True
    )
    time.sleep(0.1)

    with otlp_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

If that seems to work, but you don't see traces appear in AWS X-Ray, check the ADOT container logs. E.g. there may be a permission issue if your AWS credentials have expired:


In [ ]:
!docker logs adot-collector

### `NoopTracer`

Use the no-operation tracer when you don't want traces:


In [14]:
noop_tracer = NoopTracer()
with noop_tracer.trace("noop") as span:
    span.add_attribute("foo", "bar")

# nothing was logged

## SqliteTracer

In [ ]:
sqlite_tracer = SqliteTracer(db_path="my-conversations.db")

sqlite_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

with sqlite_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute(
        "ai.auth.context", {"principal_id": "user123"}, inheritable=True
    )
    time.sleep(0.1)

    with sqlite_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)

### `TeeTracer`

Use the `TeeTracer` to send traces to multiple tracers at once.

Note that the first tracer you add, will be the one that `get_traces()` will be delegated to. So if you want to use that method, use a tracer that supports it.

Add tracers like this:


In [15]:
tee_tracer = TeeTracer()

# E.g. the DynamoDBTracer supports get_traces(), so add that first:
tee_tracer.add_tracer(ddb_tracer)

# add_tracer() can be chained for convenience:
tee_tracer.add_tracer(human_readable_tracer).add_tracer(noop_tracer)

Then, use the `TeeTracer` as any other tracer:


In [16]:
conversation_id = Ulid().ulid
auth_context = {"principal_id": "user456"}

tee_tracer.set_context(resource_attributes={"service.name": "MyAgent"})

print("==== live traces: ====\n")

with tee_tracer.trace("parent", span_kind="SERVER") as parent_span:
    parent_span.add_attribute("ai.conversation.id", conversation_id, inheritable=True)
    parent_span.add_attribute("ai.auth.context", auth_context, inheritable=True)
    time.sleep(0.1)

    with tee_tracer.trace("child") as child_span:
        child_span.add_attribute("ai.trace.type", "tool-invocation")
        child_span.add_attribute("ai.tool.input", "Hello, world!")
        child_span.add_attribute("ai.tool.output", "World, hello!")
        time.sleep(0.1)


print("==== traces from DynamoDB: ====\n")
for trace in tee_tracer.get_traces(
    attribute_filter={
        "ai.conversation.id": conversation_id,
        "ai.auth.context": auth_context,
        "ai.subcontext.id": None
    }
):
    print(trace.as_human_readable())
    print()

==== live traces: ====

[ef04cce04e9d38285a49e58b14a0a1af/4dd28f3ac0a0794a/a254fde0f24d13f8] MyAgent INTERNAL 2025-07-04T11:12:58.821Z - child
  ai.trace.type=tool-invocation ai.conversation.id=01JZAJ7FB0TE8Y0ZX00N1SDQVV ai.auth.context=user456
         Input: Hello, world!
        Output: World, hello!

[ef04cce04e9d38285a49e58b14a0a1af/root/4dd28f3ac0a0794a] MyAgent SERVER 2025-07-04T11:12:58.721Z - parent
  ai.conversation.id=01JZAJ7FB0TE8Y0ZX00N1SDQVV ai.auth.context=user456

==== traces from DynamoDB: ====

[ef04cce04e9d38285a49e58b14a0a1af/root/4dd28f3ac0a0794a] MyAgent SERVER 2025-07-04T11:12:58.721Z - parent
  ai.conversation.id=01JZAJ7FB0TE8Y0ZX00N1SDQVV ai.auth.context=user456


[ef04cce04e9d38285a49e58b14a0a1af/4dd28f3ac0a0794a/a254fde0f24d13f8] MyAgent INTERNAL 2025-07-04T11:12:58.821Z - child
  ai.trace.type=tool-invocation ai.conversation.id=01JZAJ7FB0TE8Y0ZX00N1SDQVV ai.auth.context=user456
         Input: Hello, world!
        Output: World, hello!




### `@traced` decorator

Rather than wrapping your code inside `with` statements to add tracing, you can also you use the `@traced` decorator with your functions, to trace their execution:


In [17]:
from generative_ai_toolkit.tracer import traced

in_memory_tracer = InMemoryTracer()


@traced("parent", tracer=in_memory_tracer)
def parent_fn():
    child_fn()
    time.sleep(0.1)


@traced("child", tracer=in_memory_tracer)
def child_fn():
    time.sleep(0.1)

Now, when you execute these functions, they will be traced:


In [18]:
parent_fn()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent', span_kind='INTERNAL', trace_id='e0f2948f7705263e049e50b3eece34da', span_id='51a2cf0814efe564', parent_span_id=None, started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 83700, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 291396, tzinfo=datetime.timezone.utc), attributes={}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child', span_kind='INTERNAL', trace_id='e0f2948f7705263e049e50b3eece34da', span_id='fd9dac4ada737f76', parent_span_id='51a2cf0814efe564', started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 83827, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 187722, tzinfo=datetime.timezone.utc), attributes={}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


In order to add attributes to the trace, you can access the `current_trace` attribute for the tracer. Accessing that attribute only works within the context of a trace:


In [19]:
in_memory_tracer = InMemoryTracer()


@traced("parent", tracer=in_memory_tracer)
def parent_fn2():
    in_memory_tracer.current_trace.add_attribute("foo", "bar", inheritable=True)
    child_fn2()
    time.sleep(0.1)


@traced("child", tracer=in_memory_tracer)
def child_fn2():
    in_memory_tracer.current_trace.add_attribute("bar", "foo")
    time.sleep(0.1)


parent_fn2()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent', span_kind='INTERNAL', trace_id='d607e0eae6e39063739487f1036dc80a', span_id='dbfb6da2dcd90090', parent_span_id=None, started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 298048, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 507483, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child', span_kind='INTERNAL', trace_id='d607e0eae6e39063739487f1036dc80a', span_id='418e067946334542', parent_span_id='dbfb6da2dcd90090', started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 298133, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 402403, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


If the first argument to your function has a `tracer` attribute, you don't need to specify a `tracer` explicitly. E.g. within a class with a `tracer` attribute, you can decorate methods with `@traced`, i.e. without explicitly passing the tracer, as below:


In [20]:
in_memory_tracer = InMemoryTracer()


class MyAgent:
    def __init__(self, tracer: Tracer) -> None:
        self._tracer = tracer

    @property
    def tracer(self):
        return self._tracer

    @traced
    def parent_method(self):
        self.tracer.current_trace.add_attribute("foo", "bar", inheritable=True)
        self.child_method()
        time.sleep(0.1)

    @traced
    def child_method(self):
        self.tracer.current_trace.add_attribute("bar", "foo")
        time.sleep(0.1)


agent = MyAgent(in_memory_tracer)
agent.parent_method()

for trace in in_memory_tracer.get_traces():
    print(trace)

Trace(span_name='parent_method', span_kind='INTERNAL', trace_id='b07bf6adaf0ada642cb2cf6f7a67832a', span_id='1ebb45db1c22a743', parent_span_id=None, started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 522134, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 727517, tzinfo=datetime.timezone.utc), attributes={'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)
Trace(span_name='child_method', span_kind='INTERNAL', trace_id='b07bf6adaf0ada642cb2cf6f7a67832a', span_id='bc1460775eafbe98', parent_span_id='1ebb45db1c22a743', started_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 523157, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 4, 11, 12, 59, 623815, tzinfo=datetime.timezone.utc), attributes={'bar': 'foo', 'foo': 'bar'}, span_status='UNSET', resource_attributes={}, scope=generative-ai-toolkit@current)


### Developing your own tracer

It's easy to develop your own tracers that can be used with the Generative AI Toolkit.

In the simplest case, you inherit from `BaseTracer` and only have to implement the `persist` method:


In [21]:
from generative_ai_toolkit.tracer import BaseTracer, Trace


class MyTracer(BaseTracer):
    def persist(self, trace: Trace):
        print(trace.as_human_readable())  # This is what the `HumanReadableTracer` does


my_tracer = MyTracer()

with my_tracer.trace("span") as span:
    span.add_attribute("foo", "bar")

[224706b26a7661d201b69c4ab5426750/root/07744e71cd8fef0a] <missing service.name> INTERNAL 2025-07-04T11:12:59.742Z - span




### Trace Snapshots with `IterableTracer`

The Generative AI toolkit supports capturing intermediate trace states through a "snapshot" mechanism. This allows you to observe and analyze trace data in real-time during long-running operations, without waiting for a trace to complete.

The `IterableTracer` is specifically designed to support snapshots, making it ideal for streaming processing and real-time monitoring:


In [22]:
import threading

iterable_tracer = IterableTracer()


# Start processing traces in a separate thread
def trace_processor():
    print("Trace processor started. Waiting for traces and snapshots...")
    for trace in iterable_tracer:
        print(
            f"Received trace: {trace.span_name} ({trace.ended_at is None and 'SNAPSHOT' or 'COMPLETED'})"
        )
        print(f"  - attributes: {trace.attributes}")
        print()


processor_thread = threading.Thread(target=trace_processor)
processor_thread.daemon = True
processor_thread.start()

# Now use the tracer
with iterable_tracer.trace("long-operation") as trace:
    trace.add_attribute("status", "starting")
    trace.emit_snapshot()  # This explicitly emits a snapshot
    time.sleep(1)

    trace.add_attribute("status", "halfway")
    trace.add_attribute("progress", "50%")
    trace.emit_snapshot()  # Emit another snapshot with updated state
    time.sleep(1)

    # When the context exits, the final state will be emitted automatically
    trace.add_attribute("status", "completed")
    trace.add_attribute("progress", "100%")

# Upon calling shutdown, no new traces can be recorded.
# Traces already emitted will still be available in the iterator
iterable_tracer.shutdown()
# Once the iterator is fully consumed, the thread is done
processor_thread.join()

Trace processor started. Waiting for traces and snapshots...
Received trace: long-operation (SNAPSHOT)
  - attributes: {'status': 'starting'}

Received trace: long-operation (SNAPSHOT)
  - attributes: {'status': 'halfway', 'progress': '50%'}

Received trace: long-operation (COMPLETED)
  - attributes: {'status': 'completed', 'progress': '100%'}



#### Key Points about Snapshots

1. **Snapshot vs. Complete Trace**: A snapshot contains a copy of the trace at the moment `emit_snapshot()` is called, while the complete trace is emitted when the trace context exits.

2. **Identifying Snapshots**: You can identify snapshots by checking if `trace.ended_at` is `None`, as snapshots don't have an end time.

3. **Tracing API Support**: Only certain tracers support snapshots:

   - `IterableTracer`: The primary tracer that directly supports snapshots
   - `TeeTracer`: Will proxy snapshots to any snapshot-capable tracers in its collection

4. **Custom Tracers with Snapshot Support**: To implement snapshot support in a custom tracer:


In [23]:
from generative_ai_toolkit.tracer import BaseTracer, Trace


class MySnapshotCapableTracer(BaseTracer):
    def __init__(self):
        super().__init__()
        self.snapshot_enabled = True  # Enable snapshot support

    def persist(self, trace: Trace):
        print(f"TRACE COMPLETE: {trace.span_name} - {trace.attributes}")

    def persist_snapshot(self, trace: Trace):
        print(f"SNAPSHOT: {trace.span_name} - {trace.attributes}")


# Usage example
snapshot_tracer = MySnapshotCapableTracer()

with snapshot_tracer.trace("snapshot-demo") as trace:
    trace.add_attribute("step", "initial")
    trace.emit_snapshot()
    time.sleep(0.5)

    trace.add_attribute("step", "middle")
    trace.emit_snapshot()
    time.sleep(0.5)

    trace.add_attribute("step", "final")

SNAPSHOT: snapshot-demo - {'step': 'initial'}
SNAPSHOT: snapshot-demo - {'step': 'middle'}
TRACE COMPLETE: snapshot-demo - {'step': 'final'}


#### TeeTracer with Snapshots

You can use `TeeTracer` to send snapshots to multiple trace collection systems. Note that snapshots will only be proxied to tracers that implement the `SnapshotCapableTracer` protocol and have `snapshot_enabled=True`:


In [24]:
from generative_ai_toolkit.tracer import TeeTracer

# Create a composite tracer that supports snapshots
tee_tracer = TeeTracer()
tee_tracer.add_tracer(MySnapshotCapableTracer())  # Add our snapshot-capable tracer

# Enable snapshot support on the TeeTracer
tee_tracer.snapshot_enabled = True

with tee_tracer.trace("tee-snapshot-demo") as trace:
    trace.add_attribute("step", "initial")
    trace.emit_snapshot()  # Will be proxied to all snapshot-capable tracers
    time.sleep(0.5)

    trace.add_attribute("step", "final")

SNAPSHOT: tee-snapshot-demo - {'step': 'initial'}
TRACE COMPLETE: tee-snapshot-demo - {'step': 'final'}


#### Practical Use Cases for Snapshots

1. **Streaming Responses**: Monitor progress of streaming LLM responses in real-time
2. **Long-running Tool Calls**: Track progress of tools that take significant time to execute
3. **User Interfaces**: Build responsive UIs that update as traces evolve
4. **Debugging**: Capture intermediate states to diagnose complex issues
5. **Progress Reporting**: Show operation progress to users during lengthy operations
